In [118]:
import textstat
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import language_tool_python
import statistics
import logging

# Setup basic logging (you can configure it as needed)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def analyze_essay(essay):
    """
    Analyze various aspects of an essay for linguistic sophistication.

    :param essay: A string containing the essay text.
    :return: A dictionary with various metrics and an overall quality score.
    """
    
    tool = language_tool_python.LanguageToolPublicAPI('en-US')

    # Readability and complexity metrics
    flesch_reading_ease = textstat.flesch_reading_ease(essay)
    smog_index = textstat.smog_index(essay)
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(essay)

    # Tokenize essay into words and sentences
    words = word_tokenize(essay)
    sentences = sent_tokenize(essay)
    unique_words = len(set(words))

    # Lexical diversity
    lexical_diversity = len(set(words)) / len(words) if words else 0

    # Sentence length variability
    sentence_lengths = [len(word_tokenize(sentence)) for sentence in sentences]
    sentence_length_variability = statistics.stdev(sentence_lengths) if len(sentence_lengths) > 1 else 0

    # Grammatical mistakes
    matches = tool.check(essay)
    grammar_errors = len(matches) 

    return   {
        "flesch_reading_ease": flesch_reading_ease,
        "smog_index": smog_index,
        "flesch_kincaid_grade": flesch_kincaid_grade,
        "unique_words": unique_words,
        "lexical_diversity": lexical_diversity,
        "grammar_errors": grammar_errors,
    }
    pass


[nltk_data] Downloading package punkt to /Users/nils/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nils/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/nils/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
import pandas as pd 
gpt = pd.read_csv('all_essays_gpt.csv', sep=';')
gemini = pd.read_csv('all_essays_gemini.csv', sep=';')
llama = pd.read_csv('all_essays_llama.csv', sep=';')

In [ ]:
gpt['essay_ESL'] = gpt['essay_ESL'].astype(str)
gpt['metrics_ESL'] = gpt['essay_ESL'].apply(analyze_essay)
gpt['metrics_AAE'] = gpt['essay_from_AAE_prompt'].apply(analyze_essay)

In [115]:
llama['metrics_SAE'] = llama['essay_base'].apply(analyze_essay)
llama['metrics_ESL'] = llama['essay_ESL'].apply(analyze_essay)
llama['metrics_AAE'] = llama['essay_from_AAE_prompt'].apply(analyze_essay)

In [ ]:
if 'metrics_SAE' not in gemini.columns:
    gemini['metrics_SAE'] = [{} for _ in range(len(gemini))]


# Loop through each essay
for index, row in gemini.iterrows():
    # Check if the analysis has not been done for this row
    if not row['metrics_SAE']:
        try:
            # Analyze the essay
            essay_metrics = analyze_essay(row['essay_base'])
            # Save the analysis result to the 'metrics_ESL' column
            gemini.at[index, 'metrics_SAE'] = essay_metrics
        except Exception as e:
            # Log or print the error and continue
            print(f"Error processing essay at index {index}: {e}")

In [ ]:
if 'metrics_AAE' not in gemini.columns:
    gemini['metrics_AAE'] = [{} for _ in range(len(gemini))]


# Loop through each essay
for index, row in gemini.iterrows():
    # Check if the analysis has not been done for this row
    if not row['metrics_AAE']:
        try:
            # Analyze the essay
            essay_metrics = analyze_essay(row['essay_from_AAE_prompt'])
            # Save the analysis result to the 'metrics_ESL' column
            gemini.at[index, 'metrics_AAE'] = essay_metrics
        except Exception as e:
            # Log or print the error and continue
            print(f"Error processing essay at index {index}: {e}")


In [ ]:
gemini['metrics_ESL'] = gemini['essay_ESL'].apply(analyze_essay)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

# Convert dictionaries to DataFrame
df_SAE = pd.DataFrame(list(gemini['metrics_SAE']))
df_ESL = pd.DataFrame(list(gemini['metrics_ESL']))
df_AAE = pd.DataFrame(list(gemini['metrics_AAE']))

# Calculate means
means_SAE = df_SAE.mean()
means_ESL = df_ESL.mean()
means_AAE = df_AAE.mean()

gemini_results = pd.concat([means_SAE, means_ESL, means_AAE], axis=1, keys=['SAE', 'ESL', 'AAE'])
gemini_results.to_csv('results/results_gemini.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

# Convert dictionaries to DataFrame
df_SAE = pd.DataFrame(list(gpt['metrics_SAE']))
df_ESL = pd.DataFrame(list(gpt['metrics_ESL']))
df_AAE = pd.DataFrame(list(gpt['metrics_AAE']))

# Calculate means
means_SAE = df_SAE.mean()
means_ESL = df_ESL.mean()
means_AAE = df_AAE.mean()

gpt_results= pd.concat([means_SAE, means_ESL, means_AAE], axis=1, keys=['SAE', 'ESL', 'AAE'])
gpt_results.to_csv('results/results_gpt.csv')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

# Convert dictionaries to DataFrame
df_SAE = pd.DataFrame(list(llama['metrics_SAE']))
df_ESL = pd.DataFrame(list(llama['metrics_ESL']))
df_AAE = pd.DataFrame(list(llama['metrics_AAE']))

# Calculate means
means_SAE = df_SAE.mean()
means_ESL = df_ESL.mean()
means_AAE = df_AAE.mean()

llama_results = pd.concat([means_SAE, means_ESL, means_AAE], axis=1, keys=['SAE', 'ESL', 'AAE'])
llama_results.to_csv('results/results_llama.csv')